In [5]:
import numpy as np
import pandas as pd
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC


In [13]:
driver = webdriver.Chrome(service = Service("myenv/Lib/chromedriver.exe"))

driver.get(r"https://shopee.vn/S%E1%BA%AFc-%C4%90%E1%BA%B9p-cat.11036279?page=1")
sleep(random.randint(5,10))
#driver.find_element(By.CSS_SELECTOR,"#main button:nth-child(2)").click()
driver.find_element(By.NAME, "loginKey").send_keys("thithaovytran809")
driver.find_element(By.NAME, "password").send_keys("Thaovytran809@")
driver.find_element(By.CSS_SELECTOR, "button.DYKctS").click()
#lấy title,link,price,sale_counr
prods = driver.find_elements(By.CSS_SELECTOR, "li.col-xs-2-4")
prod_list = []
for prod in prods:
    title = prod.find_element(By.CSS_SELECTOR, "div._7V9Hys").text
    link = prod.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    price = prod.find_element(By.CSS_SELECTOR, "div.l7chvI").text[1]
    sale_count = prod.find_element(By.CSS_SELECTOR, "div.CxiTvi").text
    prod_json = {
        "title":title,
        "link":link,
        "price":price,
        "sale_count":sale_count
    }
    prod_list.append(prod_json)
    # print(prod_list)
    # break
df1 = pd.DataFrame(prod_list)
df1["index_"] = np.arange(1,len(df1)+1)

# lấy disc, or_price, thêm vào df2
or_price_list, disc_perc_list, discount_indx = [],[],[]
for i in range(1,len(df1)+1):
    try:
        or_price = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/div/div/div[4]/div[2]/section/ul/li[{}]/a/div/div/div[2]/div[2]/div[2]".format(i))
        or_price_list.append(or_price)
        disc_perc = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[4]/div[2]/section/ul/li[{}]/a/div/div/div[2]/div[2]/div[3]/div/span".format(i))
        disc_perc_list.append(disc_perc)
        discount_indx.append(i)
    except NoSuchElementException:
        pass

df2 = pd.DataFrame(list(zip(discount_indx, or_price_list, disc_perc_list)), columns=["discount_indx","or_price_list","disc_perc_list"])

df3 = df1.merge(df2, how="left", left_on="index_", right_on="discount_indx")


# lấy city thêm vào df4 
city_list, city_indx =[]
for i in range (1,len(df1)+1):
    try:
        city = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div[4]/div[2]/section/ul/li[{}]/a/div/div/div[2]/div[4]/div".format(i))
        city_list.append(city)
        city_indx.append(i)
    except NoSuchElementException:
        pass
df4 = pd.DataFrame(list(zip(city_indx, city_list)), columns=["city_indx", "city_list"])

df5 = df3.merge(df4, how="left", left_on="index_", right_on="city_indx")

# lấy catalog và rate
def catalogandrate(link):
    driver.get(link)
    catalog = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div/div[2]/div[3]/div/div[1]/div[1]/section[1]/div/div[1]/div/a[4]").text
    #catalog_list = [catalog.text for catalog in catalogs]
    rate = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[2]/div[1]/div/div[1]/div/div[2]/div[3]/div/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/span[1]").text
    #rate_list = [rate.text for rate in rates]
    return catalog, rate

catalog_list = []
rate_list = []
for link in df5["link"]:
    catalog, rate = catalogandrate(link)
    catalog_list.append(catalog)
    rate_list.append(rate)
df5["catalog_list"] = catalog_list
df5["rate_list"] = rate_list











    